##  Unsupervised KMeans image classification with the Orfeo Toolbox

The application runs the unsupervised KMeans image classification using four bands of a Sentinel-2 tile

### <a name="quicklink">Quick link

* [Objective](#objective)
* [Data](#data)
* [Service Definition](#service)
* [Parameter Definition](#parameter)
* [Runtime Parameter Definition](#runtime)
* [Workflow](#workflow)
* [License](#license)

### <a name="objective">Objective 

Apply the unsupervised KMeans image classification using the Orfeo Toolbox Python bindings to a Sentinel-2 tile 

### <a name="data">Data 

SENTINEL data products are made available systematically and free of charge to all data users including the general public, scientific and commercial users. Radar data will be delivered within an hour of reception for Near Real-Time (NRT) emergency response, within three hours for NRT priority areas and within 24 hours for systematically archived data.

The data used are Sentinel-2 Level-1C products: Top of atmosphere reflectances in fixed cartographic geometry (combined UTM projection and WGS84 ellipsoid). Level-1C images are a set of tiles of 100 sq km, each of which is approximately 500 MB. These products contain applied radiometric and geometric corrections (including orthorectification and spatial registration). 

The spectral bands of Sentinel-2 Level-1C products are:

| S-2 band                | 1   | 2   | 3   | 4   | 5   | 6   | 7   | 8   | 8a  | 9   | 10   | 11   | 12   |
|-------------------------|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|------|------|------|
| Central wavelength (nm) | 443 | 490 | 560 | 665 | 705 | 740 | 783 | 842 | 865 | 945 | 1375 | 1610 | 2190 |
| Bandwidth (nm)          | 20  | 65  | 35  | 30  | 15  | 15  | 20  | 115 | 20  | 20  | 30   | 90   | 180  |
| Spatial resolution (m)  | 60  | 10  | 10  | 10  | 20  | 20  | 20  | 10  | 20  | 60  | 60   | 20   | 20   |

### <a name="service">Service Definition

In [6]:
service = dict([('title', 'ewf-notebook-stagein-1'),
                ('abstract', 'Unsupervised KMeans image classification with the Orfeo Toolbox'),
                ('id', 'ewf_notebook_1')])

### <a name="parameter">Parameter Definition 

**Training set size**

Size of the training set (in pixels)

In [16]:
ts = dict([('id', 'ts'),
               ('value', '1000'),
               ('title', 'Training set size'),
               ('abstract', 'Size of the training set (in pixels)')])

**Number of classes**

Number of modes, which will be used to generate class membership

In [1]:
nc = dict([('id', 'nc'),
           ('value', '5'),
           ('title', 'Number of classes'),
           ('abstract', 'Number of modes, which will be used to generate class membership')])

**Maximum number of iterations**

Maximum number of iterations for the learning step

In [2]:
maxit = dict([('id', 'maxit'),
              ('value', '1000'),
              ('title', 'Maximum number of iterations'),
              ('abstract', 'Maximum number of iterations for the learning step')])

### <a name="runtime">Runtime parameter definition

**Input identifier**

This is the Sentinel-2 product identifier

In [6]:
input_identifier = 'S2A_MSIL1C_20170909T060631_N0205_R134_T42SVG_20170909T061506'

**Input reference**

This is the Sentinel-2 catalogue reference

In [5]:
input_reference = 'https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL1C_20170909T060631_N0205_R134_T42SVG_20170909T061506'

**Data path**

This path defines where the data is staged-in. 

In [5]:
data_path = '/workspace/data'

### <a name="workflow">Workflow

#### Import the packages required for processing the Sentinel-1 backscatter

In [4]:
import os
import sys
import glob

sys.path.append('/opt/OTB-6.2.0/lib/python')
sys.path.append('/opt/OTB-6.2.0/lib/libfftw3.so.3')
os.environ['OTB_APPLICATION_PATH'] = '/opt/OTB-6.2.0/lib/otb/applications'
os.environ['LD_LIBRARY_PATH'] = '/opt/OTB-6.2.0/lib'
os.environ['ITK_AUTOLOAD_PATH'] = '/opt/OTB-6.2.0/lib/otb/applications'
os.environ['GDAL_DATA'] = '/opt/anaconda/share/gdal/'
import otbApplication

#### Identify the tile to process

In [7]:
search_expression = os.path.join(data_path, input_identifier, input_identifier + '.SAFE', 'manifest.safe')

In [8]:
s2_manifest = glob.glob(search_expression)[0] 

In [9]:
cwd = os.getcwd()
granule_path = os.path.join(data_path, input_identifier, input_identifier + '.SAFE', 'GRANULE')
os.chdir(granule_path)

In [10]:
granules = [ name for name in os.listdir('.') if os.path.isdir(name)]

os.chdir(cwd)

granules

['L1C_T42SVG_A011571_20170909T061506']

In [11]:
granule_index = 0

s2_bands = []

for band in [2, 3, 4, 8]:

    search_expression = os.path.join(granule_path, granules[granule_index], 'IMG_DATA', '*_B0' + str(band) + '.jp2')
    s2_bands.append(glob.glob(search_expression)[0])

#### Concatenate the bands with OTB

In [12]:
OTB_app1 = otbApplication.Registry.CreateApplication('ConcatenateImages')

OTB_app1.SetParameterStringList('il', s2_bands)
OTB_app1.SetParameterString('out', 'concat.tif')

OTB_app1.ExecuteAndWriteOutput()

0

#### Apply the KMeans Classification OTB application

In [13]:
OTB_app2 = otbApplication.Registry.CreateApplication('KMeansClassification')

OTB_app2.SetParameterString('in', 'concat.tif')
OTB_app2.SetParameterInt('ts', ts)
OTB_app2.SetParameterInt('nc', nc)
OTB_app2.SetParameterInt('maxit', maxit)
OTB_app2.SetParameterString('out', 'classification.tif')

OTB_app2.ExecuteAndWriteOutput()

0

In [14]:
os.remove('concat.tif')

#### Produce an RGB output with a look-up table

In [15]:
file = open('classification.lut', 'w') 
 
file.write('0 51 153 255') 
file.write('1 255 255 204') 
file.write('2 229 255 204') 
file.write('3 204 255 255') 
file.write('4 255 204 255') 
 
file.close() 

In [18]:
OTB_app3 = otbApplication.Registry.CreateApplication('ColorMapping')

OTB_app3.SetParameterString('in', 'classification.tif')
OTB_app3.SetParameterString('method', 'custom')
OTB_app3.SetParameterString('method.custom.lut', 'classification.lut')
OTB_app3.SetParameterString('out', 'preview.tif')
OTB_app3.SetParameterOutputImagePixelType('out', otbApplication.ImagePixelType_uint8)

OTB_app3.ExecuteAndWriteOutput()

0

In [20]:
os.remove('classification.lut')

### <a name="license">License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.